In [ ]:
# https://www.rapidtables.com/web/color/RGB_Color.html
# https://github.com/codebrainz/color-names/blob/master/output/colors.csv

In [ ]:
import numpy as np
import cv2
import os
import pandas as pd
from matplotlib import pyplot as plt

# img = cv2.imread("../img/eardrops.jpg")
# img = cv2.imread("../img/flowers.jpg")
img = cv2.imread("../img/lego.png")

img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

plt.imshow(img)
plt.show()

In [ ]:
# K Means clustering

# Convert to float32 array of N x 3, where each row is a pixel (R, G, B)
pixels = np.float32(img.reshape(-1, 3))

K = 3
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
flags = cv2.KMEANS_RANDOM_CENTERS

_, labels, palette = cv2.kmeans(pixels, K, None, criteria, 10, flags)
_, counts = np.unique(labels, return_counts=True)

average = img.mean(axis=0).mean(axis=0)
dominant = palette[np.argmax(counts)]

In [ ]:
print(average)
print()
print(dominant)
print()
print(counts)
print()
print(palette)

In [ ]:
avg_patch = np.ones(shape=img.shape, dtype=np.uint8)*np.uint8(average)

indices = np.argsort(counts)[::-1]   
freqs = np.cumsum(np.hstack([[0], counts[indices]/float(counts.sum())]))
rows = np.int_(img.shape[0]*freqs)

dom_patch = np.zeros(shape=img.shape, dtype=np.uint8)
for i in range(len(rows) - 1):
    dom_patch[rows[i]:rows[i + 1], :, :] += np.uint8(palette[indices[i]])
    
fig, (ax0, ax1) = plt.subplots(1, 2, figsize=(12,6))
ax0.imshow(avg_patch)
ax0.set_title('Average color')
ax0.axis('off')
ax1.imshow(dom_patch)
ax1.set_title('Dominant colors')
ax1.axis('off')
plt.show(fig)

In [ ]:
# Extract top 3 colors
count_to_color = {}
for i in range(len(counts)):
    count_to_color[counts[i]] = palette[i]
# print(count_to_color)

colors_sorted = []
for key in sorted(count_to_color, reverse=True):
    colors_sorted.append(np.array(count_to_color[key]))
    print("%s: %s" % (key, count_to_color[key]))
print()
print(colors_sorted)

In [ ]:
# colors = pd.read_csv("colors_small.csv")
colors = pd.read_csv("colors_medium.csv")
# colors = pd.read_csv("colors_large.csv")
color_names = []

for color in colors_sorted:
    min_dist = np.inf
    color_name = ""
    for _, row in colors.iterrows():
        rgb_array = np.array([row["R"], row["G"], row["B"]])

        dist = np.linalg.norm(color - rgb_array)
        if dist < min_dist:
            min_dist = dist
            color_name = row["Name"]
    color_names.append(color_name)
print(color_names)